In [120]:
from phi.agent import Agent
from phi.model.google import Gemini
from phi.knowledge.pdf import PDFKnowledgeBase
from phi.vectordb.pgvector import PgVector, SearchType
from phi.tools.duckduckgo import DuckDuckGo

### To use PGVector 

In [118]:
db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"
knowledge_base = PDFKnowledgeBase(
    path="files",   #Path of pdf  files
    vector_db=PgVector(
        table_name="documents",
        db_url=db_url,
        search_type=SearchType.vector,
    ),
)

# Load the knowledge base: Comment out after first run
knowledge_base.load(upsert=True)

DEBUG    Initialized PgVector with table 'ai.documents'

INFO     Creating collection

DEBUG    Checking if table 'ai.documents' exists.

INFO     Loading knowledge base

INFO     Reading: hwawei_annual_report

...

INFO     Added 7 documents to knowledge base

INFO     Reading: RFI

DEBUG    Processing batch starting at index 0, size: 5

INFO     Upserted batch of 5 documents.

INFO     Added 5 documents to knowledge base

### To lanceDB

In [ ]:
# To use lance db

from phi.vectordb.lancedb import LanceDb
from phi.vectordb.search import SearchType

# LanceDB Vector DB
lance_vector_db = LanceDb(
    table_name="documents",
    uri="./tmp/lancedb",
    search_type=SearchType.vector,
)
lance_knowledge_base = PDFKnowledgeBase(
    path="files",
    vector_db=lance_vector_db
)
# Comment out after first run
lance_knowledge_base.load(recreate=True)

In [76]:
## To add new files

from phi.document.base import Document
from phi.document.reader.base import Reader
from phi.document.reader.pdf import PDFReader

docs=PDFReader().read("files/hwawei_annual_report.pdf")

knowledge_base.load_documents(docs)

INFO     Reading: hwawei_annual_report

INFO     Loading knowledge base

DEBUG    Creating collection

DEBUG    Checking if table 'ai.documents' exists.

DEBUG    Processing batch starting at index 0, size: 100

INFO     Inserted batch of 100 documents.

DEBUG    Processing batch starting at index 100, size: 95

INFO     Inserted batch of 95 documents.

INFO     Loaded 195 documents to knowledge base

### Create Agent 

In [122]:

rag_agent = Agent(
    model=Gemini(id="gemini-2.0-flash-exp"),
    knowledge=lance_knowledge_base,    # or use knowledge_base
    tools=[DuckDuckGo],
    # Add a tool to search the knowledge base which enables agentic RAG.
    search_knowledge=True,
    # Add a tool to read chat history.
    read_chat_history=True,
    
    show_tool_calls=True,
    markdown=True,
    debug_mode=False,
)


DEBUG    *********** Agent ID: 765fd557-7e2e-4b14-8c80-56068760c99c ***********

DEBUG    *********** Session ID: 0be4602c-e017-4e42-8f2d-46d19696510e ***********

### Run Agent

In [116]:
rag_agent.print_response("Show some information about hwawei", stream=True)

DEBUG    *********** Agent Run Start: c524adf5-7312-4894-ae2c-95d61fa3c3f5 ***********

DEBUG    ---------- Gemini Response Start ----------

DEBUG    ============== system ==============

DEBUG    ## Instructions                                                                                           
         Use markdown to format your answers.

DEBUG    ============== user ==============

...

DEBUG    ---------- Gemini Response End ----------

DEBUG    Added 2 Messages to AgentMemory

DEBUG    Added AgentRun to AgentMemory

DEBUG    --**-- Logging Agent Run

DEBUG    *********** Agent Run End: c524adf5-7312-4894-ae2c-95d61fa3c3f5 ***********